# Transfer learning tests
Eamon Whalen

In [1]:
import sys
import os
import glob
import numpy as np
import pandas as pd
import altair as alt

from gcnSurrogate.models.feastnetSurrogateModel import FeaStNet
from gcnSurrogate.models.pointRegressorSurrogateModel import PointRegressor
from gcnSurrogate.readers.loadConmechGraphs import loadConmechGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

## 1. Load test data

In [2]:
# dataDir = "data/2D_Truss_v1.3/gh/"
# testFile = os.path.join(dataDir, 'design_9_N_1000.csv')
# allGraphsUnfiltered = loadGhGraphs(testFile, NUM_DV=5)
dataDir = "data/endLoadsv1.0/conmech/"
testDir = os.path.join(dataDir, 'design_7_N_1000/')
allGraphsUnfiltered = loadConmechGraphs(testDir)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.476726
std,4.870080
min,0.013036
25%,0.045080
50%,0.089405
75%,0.187214
max,102.305313


In [3]:
testData = filterbyDisp(allGraphsUnfiltered, 0.9)
maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in testData]
source = pd.DataFrame(maxes, columns=['maxes'])
maxDispCutoff = source.max().item()
source.describe()

,maxes
count,900.000000
mean,0.107378
std,0.088163
min,0.013036
25%,0.042086
50%,0.079557
75%,0.139971
max,0.411479


## 2. Load pre-train data

In [4]:
# pretrainFiles = glob.glob(os.path.join(dataDir, '*1000.csv'))
# pretrainFiles.remove(testFile)

# allPretrainGraphs = []
# for pretrainFile in pretrainFiles:
#     designName = pretrainDir.split('/')[-1].split('_N')[0]
#     print(f'loading {designName}')
#     graphsUnfiltered = loadGhGraphs(pretrainFile, NUM_DV=5)
#     graphs = filterbyDisp(graphsUnfiltered, 0.9)
#     allPretrainGraphs.extend(graphs)

# print(f'loaded {len(allPretrainGraphs)} pretraining graphs')
# pretrainData, pretrainValData, _ = partitionGraphList(allPretrainGraphs, testSize=0.0, valSize=0.15)

In [5]:
pretrainDirs = glob.glob('data/2D_Truss_v1.3/conmech/design_7_N_1000*/')
# pretrainDirs.remove(testDir)

allPretrainGraphs = []
for pretrainDir in pretrainDirs:
    designName = pretrainDir.split('/')[-2].split('_N')[0]
    print(f'loading {designName}')
    graphsUnfiltered = loadConmechGraphs(pretrainDir)
    graphs = filterbyDisp(graphsUnfiltered, 0.9)
    allPretrainGraphs.extend(graphs)

print(f'loaded {len(allPretrainGraphs)} pretraining graphs')
pretrainData, pretrainValData, _ = partitionGraphList(allPretrainGraphs, testSize=0.0, valSize=0.15)

loading design_7
loaded 900 pretraining graphs


## 3. Initial training

In [6]:
saveDir = 'results/transferLrn_endloads_des7_01/'
epochs = 100
ptrGcn = FeaStNet()
history = ptrGcn.trainModel(pretrainData, pretrainValData, 
                            epochs=epochs,
                            saveDir=saveDir+f'preTrain/gcn/')

ptrGcnCheckptFile = ptrGcn.checkptFile
plotHistory(history)

epoch: 0   trainLoss: 8.8630e-01   valLoss:8.6201e-01  time: 2.17e+00
epoch: 1   trainLoss: 6.6728e-01   valLoss:8.4085e-01  time: 1.88e+00
epoch: 2   trainLoss: 5.3656e-01   valLoss:9.0043e-01  time: 1.86e+00
epoch: 3   trainLoss: 4.4838e-01   valLoss:1.2044e+00  time: 1.85e+00
epoch: 4   trainLoss: 3.7129e-01   valLoss:1.5456e+00  time: 1.84e+00
epoch: 5   trainLoss: 3.2526e-01   valLoss:2.1799e+00  time: 1.90e+00
epoch: 6   trainLoss: 3.0470e-01   valLoss:2.4997e+00  time: 1.91e+00
epoch: 7   trainLoss: 2.5104e-01   valLoss:2.1932e+00  time: 1.86e+00
epoch: 8   trainLoss: 2.3973e-01   valLoss:1.3317e+00  time: 1.87e+00
epoch: 9   trainLoss: 1.9596e-01   valLoss:8.6915e-01  time: 1.86e+00
epoch: 10   trainLoss: 1.5860e-01   valLoss:5.6002e-01  time: 1.85e+00
epoch: 11   trainLoss: 1.5497e-01   valLoss:1.1216e+00  time: 1.87e+00
epoch: 12   trainLoss: 1.4554e-01   valLoss:5.9140e-01  time: 1.86e+00
epoch: 13   trainLoss: 1.3024e-01   valLoss:6.9048e-01  time: 1.86e+00
epoch: 14   trai

alt.Chart(...)

In [7]:
trainRes = ptrGcn.testModel(pretrainData)
testRes = ptrGcn.testModel(testData) # unseen topology
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000006,0.001660,0.086164,0.878713,0.952191,0.823229,0.00000
test,0.004393,0.043491,0.709887,-0.497045,0.000000,-1.183517,-3.72334


## 4. Transfer learning study

In [8]:
trainDataDirs = glob.glob(os.path.join(dataDir, 'design_7*/'))
trainDataDirs.remove(testDir)

allResults = []
for trainDataDir in trainDataDirs:
    trainDataUnfiltered = loadConmechGraphs(trainDataDir)
    trainData = filterbyDispValue(trainDataUnfiltered, maxDispCutoff)
    trainSize = len(trainData)
    print(f'loaded train set of size {trainSize}')
    
    
    ### fresh neural network ###
    gcn = FeaStNet()
    history = gcn.trainModel(trainData, trainData, 
                         epochs=epochs, 
                         saveDir=saveDir+f'{trainSize:05}/gcn/')
    
    trainRes = gcn.testModel(trainData)
    trainRes['Model'] = 'Fresh'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = gcn.testModel(testData)
    testRes['Model'] = 'Fresh'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)

    
    ### transfer learning ###
    ptrGcn = FeaStNet()
    history = gcn.trainModel(trainData, trainData, 
                             restartFile=ptrGcnCheckptFile,
                             epochs=epochs, 
                             saveDir=saveDir+f'{trainSize:05}/ptrGcn/')
    
    trainRes = gcn.testModel(trainData)
    trainRes['Model'] = 'Transfer learning'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = gcn.testModel(testData)
    testRes['Model'] = 'Transfer learning'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)
    
    
    ### random forest ###
    rf = PointRegressor('Random Forest')
    rf.trainModel(trainData, trainData, 
                     saveDir=saveDir+f'{trainSize:05}/rf/')

    trainRes = rf.testModel(trainData)
    trainRes['Model'] = 'Random Forest'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = rf.testModel(testData)
    testRes['Model'] = 'Random Forest'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)
    
    
pd.DataFrame(allResults)

loaded train set of size 915
epoch: 0   trainLoss: 8.2305e-01   valLoss:9.2133e-01  time: 7.88e+00
epoch: 1   trainLoss: 5.0989e-01   valLoss:9.5012e-01  time: 7.83e+00
epoch: 2   trainLoss: 3.6870e-01   valLoss:1.0310e+00  time: 7.90e+00
epoch: 3   trainLoss: 2.7486e-01   valLoss:1.1288e+00  time: 7.82e+00
epoch: 4   trainLoss: 2.0767e-01   valLoss:1.3173e+00  time: 7.85e+00
epoch: 5   trainLoss: 1.7632e-01   valLoss:7.4724e-01  time: 8.30e+00
epoch: 6   trainLoss: 1.6974e-01   valLoss:3.6580e-01  time: 7.88e+00
epoch: 7   trainLoss: 1.2860e-01   valLoss:2.8667e-01  time: 7.99e+00
epoch: 8   trainLoss: 1.0812e-01   valLoss:2.6082e-01  time: 8.06e+00
epoch: 9   trainLoss: 9.4504e-02   valLoss:2.4619e-01  time: 8.20e+00
epoch: 10   trainLoss: 8.1976e-02   valLoss:2.0037e-01  time: 7.91e+00
epoch: 11   trainLoss: 7.8250e-02   valLoss:2.1785e-01  time: 8.22e+00
epoch: 12   trainLoss: 7.7427e-02   valLoss:2.8085e-01  time: 8.13e+00
epoch: 13   trainLoss: 6.6617e-02   valLoss:2.3471e-01  ti

epoch: 15   trainLoss: 1.8244e+00   valLoss:2.0599e+01  time: 7.56e+00
epoch: 16   trainLoss: 1.9154e+00   valLoss:6.4602e+00  time: 7.59e+00
epoch: 17   trainLoss: 1.6011e+00   valLoss:6.7002e+00  time: 7.35e+00
epoch: 18   trainLoss: 1.8178e+00   valLoss:4.2328e+00  time: 7.39e+00
epoch: 19   trainLoss: 1.3971e+00   valLoss:5.1665e+00  time: 7.47e+00
epoch: 20   trainLoss: 1.2661e+00   valLoss:6.6926e+00  time: 7.43e+00
epoch: 21   trainLoss: 1.2189e+00   valLoss:2.2483e+00  time: 7.38e+00
epoch: 22   trainLoss: 1.1638e+00   valLoss:2.1929e+00  time: 7.49e+00
epoch: 23   trainLoss: 1.3899e+00   valLoss:4.4565e+00  time: 7.38e+00
epoch: 24   trainLoss: 1.2264e+00   valLoss:1.8534e+00  time: 7.35e+00
epoch: 25   trainLoss: 9.6995e-01   valLoss:3.0094e+00  time: 7.65e+00
epoch: 26   trainLoss: 9.7075e-01   valLoss:2.9440e+00  time: 7.40e+00
epoch: 27   trainLoss: 1.2304e+00   valLoss:1.0654e+01  time: 7.45e+00
epoch: 28   trainLoss: 9.4523e-01   valLoss:2.1100e+00  time: 7.39e+00
epoch:

epoch: 30   trainLoss: 6.3618e-02   valLoss:1.6582e+00  time: 1.55e+00
epoch: 31   trainLoss: 6.6581e-02   valLoss:1.5642e+00  time: 1.55e+00
epoch: 32   trainLoss: 7.1668e-02   valLoss:1.7114e+00  time: 1.52e+00
epoch: 33   trainLoss: 6.3438e-02   valLoss:1.3648e+00  time: 1.52e+00
epoch: 34   trainLoss: 4.8213e-02   valLoss:1.2689e+00  time: 1.49e+00
epoch: 35   trainLoss: 6.0739e-02   valLoss:1.3861e+00  time: 1.49e+00
epoch: 36   trainLoss: 6.0462e-02   valLoss:1.3675e+00  time: 1.49e+00
epoch: 37   trainLoss: 4.7917e-02   valLoss:1.5128e+00  time: 1.56e+00
epoch: 38   trainLoss: 4.9866e-02   valLoss:1.7106e+00  time: 1.57e+00
epoch: 39   trainLoss: 3.9271e-02   valLoss:1.5482e+00  time: 1.57e+00
epoch: 40   trainLoss: 4.4066e-02   valLoss:1.5005e+00  time: 1.49e+00
epoch: 41   trainLoss: 3.3576e-02   valLoss:1.5621e+00  time: 1.54e+00
epoch: 42   trainLoss: 3.5388e-02   valLoss:1.2697e+00  time: 1.49e+00
epoch: 43   trainLoss: 3.2727e-02   valLoss:1.3754e+00  time: 1.49e+00
epoch:

epoch: 45   trainLoss: 6.4518e-01   valLoss:1.5558e+00  time: 1.50e+00
epoch: 46   trainLoss: 6.6394e-01   valLoss:1.6361e+00  time: 1.51e+00
epoch: 47   trainLoss: 6.1287e-01   valLoss:1.4671e+00  time: 1.57e+00
epoch: 48   trainLoss: 5.8264e-01   valLoss:1.5587e+00  time: 1.55e+00
epoch: 49   trainLoss: 5.1087e-01   valLoss:1.9051e+00  time: 1.56e+00
epoch: 50   trainLoss: 5.2403e-01   valLoss:1.8243e+00  time: 1.55e+00
epoch: 51   trainLoss: 4.9104e-01   valLoss:1.8925e+00  time: 1.56e+00
epoch: 52   trainLoss: 4.6794e-01   valLoss:2.2327e+00  time: 1.55e+00
epoch: 53   trainLoss: 4.4604e-01   valLoss:2.4958e+00  time: 1.55e+00
epoch: 54   trainLoss: 4.0803e-01   valLoss:2.2439e+00  time: 1.57e+00
epoch: 55   trainLoss: 4.0454e-01   valLoss:2.2010e+00  time: 1.57e+00
epoch: 56   trainLoss: 3.9952e-01   valLoss:3.3729e+00  time: 1.53e+00
epoch: 57   trainLoss: 3.9075e-01   valLoss:3.7844e+00  time: 1.54e+00
epoch: 58   trainLoss: 3.6631e-01   valLoss:5.0956e+00  time: 1.55e+00
epoch:

epoch: 60   trainLoss: 1.6445e-03   valLoss:1.3249e-01  time: 4.17e-01
epoch: 61   trainLoss: 1.7040e-03   valLoss:1.4450e-01  time: 4.15e-01
epoch: 62   trainLoss: 1.7737e-03   valLoss:1.3136e-01  time: 3.90e-01
epoch: 63   trainLoss: 3.0360e-03   valLoss:1.5058e-01  time: 3.85e-01
epoch: 64   trainLoss: 6.7398e-03   valLoss:1.1914e-01  time: 3.83e-01
epoch: 65   trainLoss: 1.4142e-02   valLoss:1.8546e-01  time: 3.85e-01
epoch: 66   trainLoss: 2.2467e-02   valLoss:1.0320e-01  time: 4.00e-01
epoch: 67   trainLoss: 1.0302e-02   valLoss:1.1035e-01  time: 3.89e-01
epoch: 68   trainLoss: 6.2341e-03   valLoss:1.1758e-01  time: 4.20e-01
epoch: 69   trainLoss: 1.1493e-02   valLoss:9.0771e-02  time: 4.11e-01
epoch: 70   trainLoss: 3.7581e-03   valLoss:7.8927e-02  time: 4.15e-01
epoch: 71   trainLoss: 9.2247e-03   valLoss:7.9036e-02  time: 4.13e-01
epoch: 72   trainLoss: 2.8128e-03   valLoss:9.8772e-02  time: 4.11e-01
epoch: 73   trainLoss: 7.2053e-03   valLoss:8.2085e-02  time: 4.17e-01
epoch:

...

epoch: 51   trainLoss: 3.5904e-01   valLoss:6.9845e-01  time: 1.84e-01
epoch: 52   trainLoss: 3.4725e-01   valLoss:7.0307e-01  time: 1.88e-01
epoch: 53   trainLoss: 3.3774e-01   valLoss:6.4768e-01  time: 1.83e-01
epoch: 54   trainLoss: 3.0794e-01   valLoss:6.4136e-01  time: 1.85e-01
epoch: 55   trainLoss: 2.9666e-01   valLoss:6.8533e-01  time: 1.79e-01
epoch: 56   trainLoss: 3.0818e-01   valLoss:4.7385e-01  time: 1.79e-01
epoch: 57   trainLoss: 2.8505e-01   valLoss:6.4952e-01  time: 1.74e-01
epoch: 58   trainLoss: 2.6477e-01   valLoss:7.1349e-01  time: 1.75e-01
epoch: 59   trainLoss: 2.5783e-01   valLoss:4.7538e-01  time: 1.74e-01
epoch: 60   trainLoss: 2.6574e-01   valLoss:5.5897e-01  time: 1.75e-01
epoch: 61   trainLoss: 3.0464e-01   valLoss:5.5957e-01  time: 1.73e-01
epoch: 62   trainLoss: 2.6286e-01   valLoss:5.4200e-01  time: 1.85e-01
epoch: 63   trainLoss: 2.5643e-01   valLoss:8.2482e-01  time: 1.90e-01
epoch: 64   trainLoss: 2.6296e-01   valLoss:5.9931e-01  time: 1.87e-01
epoch:

epoch: 69   trainLoss: 9.1062e-05   valLoss:8.1257e-02  time: 5.09e-02
epoch: 70   trainLoss: 8.4850e-05   valLoss:8.1160e-02  time: 4.95e-02
epoch: 71   trainLoss: 8.2007e-05   valLoss:8.1976e-02  time: 5.00e-02
epoch: 72   trainLoss: 7.8732e-05   valLoss:8.1256e-02  time: 4.93e-02
epoch: 73   trainLoss: 8.0517e-05   valLoss:8.2661e-02  time: 4.90e-02
epoch: 74   trainLoss: 7.6344e-05   valLoss:8.0686e-02  time: 4.98e-02
epoch: 75   trainLoss: 9.0884e-05   valLoss:8.2566e-02  time: 5.86e-02
epoch: 76   trainLoss: 1.2991e-04   valLoss:7.8268e-02  time: 5.85e-02
epoch: 77   trainLoss: 2.5883e-04   valLoss:8.2947e-02  time: 5.89e-02
epoch: 78   trainLoss: 5.2321e-04   valLoss:7.3389e-02  time: 5.60e-02
epoch: 79   trainLoss: 6.5966e-04   valLoss:7.8967e-02  time: 4.95e-02
epoch: 80   trainLoss: 3.5839e-04   valLoss:7.5238e-02  time: 4.92e-02
epoch: 81   trainLoss: 9.9657e-05   valLoss:7.0166e-02  time: 5.04e-02
epoch: 82   trainLoss: 3.4369e-04   valLoss:7.3991e-02  time: 4.99e-02
epoch:

epoch: 86   trainLoss: 6.8440e-02   valLoss:7.4346e-01  time: 5.26e-02
epoch: 87   trainLoss: 6.6129e-02   valLoss:7.5190e-01  time: 5.32e-02
epoch: 88   trainLoss: 6.3975e-02   valLoss:7.1794e-01  time: 5.41e-02
epoch: 89   trainLoss: 6.1713e-02   valLoss:7.1245e-01  time: 5.34e-02
epoch: 90   trainLoss: 5.9547e-02   valLoss:7.2301e-01  time: 5.48e-02
epoch: 91   trainLoss: 5.7642e-02   valLoss:7.0205e-01  time: 5.42e-02
epoch: 92   trainLoss: 5.5730e-02   valLoss:7.1862e-01  time: 5.22e-02
epoch: 93   trainLoss: 5.3980e-02   valLoss:7.2304e-01  time: 5.41e-02
epoch: 94   trainLoss: 5.2379e-02   valLoss:7.3164e-01  time: 5.19e-02
epoch: 95   trainLoss: 5.1155e-02   valLoss:7.2640e-01  time: 5.06e-02
epoch: 96   trainLoss: 5.0292e-02   valLoss:7.0233e-01  time: 5.08e-02
epoch: 97   trainLoss: 5.0913e-02   valLoss:7.0901e-01  time: 5.00e-02
epoch: 98   trainLoss: 5.5067e-02   valLoss:7.1196e-01  time: 5.12e-02
epoch: 99   trainLoss: 7.1157e-02   valLoss:6.8462e-01  time: 5.14e-02
loadin

loading restart file
epoch: 0   trainLoss: 2.9688e+01   valLoss:2.8614e+01  time: 4.24e+00
epoch: 1   trainLoss: 2.5261e+01   valLoss:2.9458e+01  time: 4.30e+00
epoch: 2   trainLoss: 2.2552e+01   valLoss:2.7977e+01  time: 4.08e+00
epoch: 3   trainLoss: 1.9697e+01   valLoss:2.4239e+01  time: 3.83e+00
epoch: 4   trainLoss: 1.7220e+01   valLoss:1.9915e+01  time: 3.79e+00
epoch: 5   trainLoss: 1.5094e+01   valLoss:1.8555e+01  time: 4.21e+00
epoch: 6   trainLoss: 1.3384e+01   valLoss:2.1172e+01  time: 3.61e+00
epoch: 7   trainLoss: 1.1171e+01   valLoss:1.9087e+01  time: 3.80e+00
epoch: 8   trainLoss: 9.5422e+00   valLoss:1.6795e+01  time: 3.61e+00
epoch: 9   trainLoss: 7.8521e+00   valLoss:1.3043e+01  time: 3.63e+00
epoch: 10   trainLoss: 6.3766e+00   valLoss:1.0171e+01  time: 3.80e+00
epoch: 11   trainLoss: 6.2081e+00   valLoss:8.7674e+00  time: 3.81e+00
epoch: 12   trainLoss: 5.0874e+00   valLoss:4.6907e+00  time: 3.80e+00
epoch: 13   trainLoss: 4.1239e+00   valLoss:4.8782e+00  time: 3.65

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2,Model,Set,Train Size
0,0.000066,0.005145,0.116020,0.969969,0.970118,0.859786,0.000000,Fresh,Train,915
1,0.000126,0.005909,0.123098,0.930629,0.947796,0.830341,0.000000,Fresh,Test,915
2,0.000060,0.004432,0.090039,0.946155,0.985602,0.867077,0.000000,Transfer learning,Train,915
3,0.000102,0.005248,0.096943,0.921676,0.969216,0.844847,0.000000,Transfer learning,Test,915
4,0.000029,0.002645,0.045589,0.982229,1.000000,0.984872,0.971596,Random Forest,Train,915
5,0.000191,0.006919,0.108247,0.883445,1.000000,0.906463,0.852468,Random Forest,Test,915
6,0.001568,0.028773,0.404843,-0.062298,0.636615,-0.099011,-1.363379,Fresh,Train,185
7,0.001534,0.028375,0.401506,0.128784,0.679991,0.024153,-1.273531,Fresh,Test,185
8,0.000051,0.004035,0.103450,0.988410,0.996774,0.858695,0.000000,Transfer learning,Train,185
9,0.000169,0.006738,0.134394,0.894760,0.953532,0.806173,0.000000,Transfer learning,Test,185


In [9]:
# df = pd.DataFrame(allResults)
df = pd.read_csv('results/transferLrn_endloads_des7_01/testResults.csv')
df

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2,Model,Set,Train Size
0,0.000066,0.005145,0.116020,0.969969,0.970118,0.859786,0.000000,Fresh,Train,915
1,0.000126,0.005909,0.123098,0.930629,0.947796,0.830341,0.000000,Fresh,Test,915
2,0.000060,0.004432,0.090039,0.946155,0.985602,0.867077,0.000000,Transfer learning,Train,915
3,0.000102,0.005248,0.096943,0.921676,0.969216,0.844847,0.000000,Transfer learning,Test,915
4,0.000029,0.002645,0.045589,0.982229,1.000000,0.984872,0.971596,Random Forest,Train,915
5,0.000191,0.006919,0.108247,0.883445,1.000000,0.906463,0.852468,Random Forest,Test,915
6,0.001568,0.028773,0.404843,-0.062298,0.636615,-0.099011,-1.363379,Fresh,Train,185
7,0.001534,0.028375,0.401506,0.128784,0.679991,0.024153,-1.273531,Fresh,Test,185
8,0.000051,0.004035,0.103450,0.988410,0.996774,0.858695,0.000000,Transfer learning,Train,185
9,0.000169,0.006738,0.134394,0.894760,0.953532,0.806173,0.000000,Transfer learning,Test,185


In [10]:
df = df.replace('Fresh', 'GCN')
df = df.replace('Transfer learning', 'GCN with transfer learning')

In [12]:
alt.Chart(df[df.Set=='Test']).mark_circle().encode(
    x=alt.X('Train Size:Q', scale=alt.Scale(type='log')),
    y=alt.Y('mse:Q', title='MSE', axis=alt.Axis(format='.1e')),
    color='Model',
    tooltip=['Model', 'mse']
).properties(width=400, height=200, title='Transfer learning accross load cases')

alt.Chart(...)